 Author: Lourde Hajjar

The notebook evaluates the performance of different activation functions on the focused dataset using a neural network architecture. The results were analyzed in terms of accuracy, precision, recall, and F1-score. Among the tested activation functions (ReLU, Sigmoid, Tanh, and ELU), ReLU produced the best results with an accuracy of 0.7923, precision of 0.8026, recall of 0.7923, and an F1-score of 0.7886. 

These metrics indicate that ReLU is the most effective activation function for this specific dataset and architecture.

## Evaluation of Different Activation Functions

## Import Libraries

In [8]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder

## Apply CNN with different Activation Functions

In [ ]:

data_dir = r"C:\Users\lourd\OneDrive\Desktop\coursework\datasets\3_image\processed\3_foc_b"


images, labels = [], []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if os.path.isdir(label_dir):
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = load_img(img_path, target_size=(256, 256), color_mode="grayscale")
            img_array = img_to_array(img) / 255.0  
            images.append(img_array)
            labels.append(label)

images = np.array(images).reshape(-1, 256, 256, 1)  
labels = np.array(labels)


label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


x_train, x_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# Apply Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(x_train)

# Assign different activation functions
activation_functions = ['relu', 'sigmoid', 'tanh', 'elu']


results = {
    'Activation Function': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': []
}


def create_model(activation):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation=activation, input_shape=(256, 256, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation=activation),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=activation),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    return model


for activation in activation_functions:
    print(f"Training with activation function: {activation}")
    

    model = create_model(activation)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        datagen.flow(x_train, y_train, batch_size=16),
        validation_data=(x_test, y_test),
        epochs=30,  
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
        verbose=0
    )
    
    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    y_pred_prob = model.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Calculate metrics
    accuracy = test_acc
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Store results
    results['Activation Function'].append(activation)
    results['Accuracy'].append(accuracy)
    results['Precision'].append(precision)
    results['Recall'].append(recall)
    results['F1-Score'].append(f1)


results_df = pd.DataFrame(results)
print("\nResults Summary:")
print(results_df)


Training with activation function: relu
11/11 [==============================] - 2s 132ms/step
Training with activation function: sigmoid
11/11 [==============================] - 2s 158ms/step
Training with activation function: tanh
11/11 [==============================] - 2s 148ms/step
Training with activation function: elu
11/11 [==============================] - 2s 144ms/step

Results Summary:
  Activation Function  Accuracy  Precision    Recall  F1-Score
0                relu  0.792285   0.802644  0.792285  0.788643
1             sigmoid  0.697329   0.729109  0.697329  0.695978
2                tanh  0.649852   0.671807  0.649852  0.604441
3                 elu  0.738872   0.767737  0.738872  0.732296
